# Inference in Pyro

**This Tutorial is adapted from [https://pyro.ai/examples/intro_part_ii.html](https://pyro.ai/examples/intro_part_ii.html)*

Much of modern machine learning can be cast as approximate inference and expressed succinctly in a language like Pyro. To motivate the rest of this tutorial, let's build a generative model for a simple physical problem so that we can use Pyro's inference machinery to solve it. However, we will first import the required modules for this tutorial:

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

Suppose we are trying to figure out how much something weighs ($W$), but the scale we're using is unreliable and gives slightly different answers every time we weigh the same object. We could try to compensate for this variability by integrating the noisy measurement ($M$) information with a guess ($G$) based on some prior knowledge about the object, like its density or material properties. The following model encodes this process:

$W \,|\,G \sim \mathcal{N}(G, 1)$

$M \,|\, W,\, G\, \sim \mathcal{N}(W,\frac{3}{4})$

Note that this is a model not only for our belief over weight, but also for the result of taking a measurement of it. The model corresponds to the following stochastic function:

In [6]:
def scale(guess):
    
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    
    return pyro.sample("measurement", dist.Normal(weight, 0.75))

## Conditioning

**The real utility of probabilistic programming is in the ability to condition generative models on observed data and infer the latent factors that might have produced that data.** In Pyro, we separate the expression of conditioning from its evaluation via inference, making it possible to write a model once and condition it on many different observations. Pyro supports constraining a model’s internal sample statements to be equal to a given set of observations.

Consider the scale example once again. Suppose we want to sample from the distribution of weight given input guess = 8.5, but now we have observed that measurement == 9.5. That is, we wish to infer the distribution:

$W\,|\,M, G \sim ?$

Pyro provides the function **pyro.condition** to allow us to constrain the values of sample statements. **pyro.condition** is a *higher-order function* that takes a *model* and a *dictionary of observations* and **returns a new model that has the same input and output signatures but always uses the given values at observed sample** statements:

In [7]:
conditioned_scale = pyro.condition(scale, data={"measurement": 9.5})

Because it behaves just like an ordinary Python function, conditioning can be deferred or parametrized with Python's **lambda** or **def**:

In [8]:
def deferred_conditioned_scale(measurement, guess):
    
    return pyro.condition(scale, data={"measurement": measurement})(guess)

In some cases it might be more convenient to pass observations directly to individual **pyro.sample** statements instead of using **pyro.condition**. The optional obs keyword argument is reserved by **pyro.sample** for that purpose:

In [9]:
def scale_obs(guess):  # equivalent to conditioned_scale above
    
    weight = pyro.sample("weight", dist.Normal(guess, 1.))
     # here we condition on measurement == 9.5
    
    return pyro.sample("measurement", dist.Normal(weight, 1.), obs=9.5)

Finally, in addition to **pyro.condition** for incorporating observations, Pyro also contains **pyro.do**, an implementation of Pearl's do-operator used for causal inference with an identical interface to **pyro.condition**. **condition** and **do** can be mixed and composed freely, making Pyro a powerful tool for model-based causal inference.

## Flexible Approximate Inference With Guide Functions

Let us return to **conditioned_scale**. Now that we have conditioned on an observation of measurement, we can use Pyro's approximate inference algorithms to estimate the distribution over weight given guess and measurement == data.

Inference algorithms in Pyro, such as **pyro.infer.SVI**, allow us to use arbitrary stochastic functions, which we will call guide functions or guides, as approximate posterior distributions. Guide functions must satisfy these two criteria to be valid approximations for a particular model: 

1. All unobserved (i.e., not conditioned) sample statements that appear in the model appear in the guide. 

2. The guide has the same input signature as the model (i.e., takes the same arguments)